
# Time : 1 hour

# Problem Statement

### A student is taking a cryptography class and has found anagrams to be very useful. Two strings are anagrams of each other if the first string's letters can be rearranged to form the second string. In other words, both strings must contain the same exact letters in the same exact frequency. For example, bacdc and dcbac are anagrams, but bacdc and dcbad are not.

### The student decides on an encryption scheme that involves two large strings. The encryption is dependent on the minimum number of character deletions required to make the two strings anagrams. You need to determine this number.

### Given two strings, $a$ and $b$, that may or may not be of the same length, determine the minimum number of character deletions required to make $a$ and $b$ anagrams. Any characters can be deleted from either of the strings. The strings $a$ and $b$ consist of lowercase English alphabets.

**Example**

<code>a = 'cde'</code>

<code>b = 'dcf' </code>

### Delete $e$ from $a$ and $f$ from $b$ so that the remaining strings are $cd$ and $dc$ which are anagrams. This takes 2 character deletions.

## **Function Description**

### Create a <code>makeAnagram</code> function below.

### **Inputs:**

### string a: a string

### string b: another string

### **Output:**

### int: the minimum total characters that must be deleted

# Solution

In [ ]:
from collections import Counter

def makeAnagram(a, b):
    a_counter = Counter(a)
    b_counter = Counter(b)

    deletions = 0
    all_char = set(a_counter.keys()).union(b_counter.keys())
    for char in all_char:
      deletions += abs(a_counter[char] - b_counter[char])

    return deletions


In [ ]:
a= 'cde'
b= 'dcf'
print(makeAnagram(a,b))

2
